INITIAL IMPORTS

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import warnings
import time
warnings.filterwarnings("ignore")
from datetime import date
import re
import requests
from scrapy.selector import Selector
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
myheaders = {'User-Agent': user_agent} 

ModuleNotFoundError: No module named 'pandas'

EVERY GAME AND GAMEID

In [3]:
#strip team names and IDs from ESPN

url = 'https://www.espn.com/mens-college-basketball/teams'
reqs = requests.get(url,headers=myheaders)
soup = BeautifulSoup(reqs.text, 'html.parser')
urls = []
teamnamelist = []
teamidlist = []
for link in soup.find_all('a'):
    newlink = link.get('href')
    newlink = str(newlink)
    string = '/team/_/id/'
    if string in newlink:
        teamname = newlink.split('/')[-1]
        teamname = re.sub("-", " ", teamname)
        teamname = ' '.join(elem.capitalize() for elem in teamname.split())
        teamnamelist.append(teamname)
        teamid = newlink.split('/')[-2]
        teamidlist.append(teamid)
        print(f'team:{teamname} ---- id: {teamid}')

teamnamedf = pd.DataFrame(teamnamelist).reset_index()
teamidlist = pd.DataFrame(teamidlist).reset_index()
df = pd.merge(teamnamedf,teamidlist,on='index')
df = df.drop(columns='index').rename(columns = {'0_x':'teamname','0_y':'id'})
teamnameiddf = df.drop_duplicates().reset_index().drop(columns='index')
display(teamnameiddf)
teamnameiddf['rosterlist'] = ''
rosterlist = []
fullnamerosterlist = []
rosterlistdictlist = []
for teamid in teamnameiddf['id']:
    df = pd.read_html(f'https://www.espn.com/mens-college-basketball/team/roster/_/id/{teamid}')
    df = df[0].drop(columns='Unnamed: 0').reset_index()
    namelist = []
    fullnamelist = []
    i = 0
    rosterlistdict = {}
    for name in df['Name']:
        pattern = r'[0-9]'
        name = re.sub(pattern, '', name)
        fullname = name
        fullnamelist.append(name)
        #get the first and last name of the player
        firstname = name.split(' ')[0]
        lastname = name.split(' ')[-1]
        #get the first letter of the first name
        lst = []
        for letter in firstname:
            lst.append(letter)
            firstinitial = lst[0]

        player = firstinitial + '. ' + lastname
        player = player.split(' ')[0] + ' ' + player.split(' ')[1]
        player = player.split(' ')[1] + ', ' + player.split(' ')[0]
        player = player.split('.')[0]
        namelist.append(player)
        rosterlistdict[player] = fullname
    rosterlist.append(namelist)
    fullnamerosterlist.append(fullnamelist)
    rosterlistdictlist.append(rosterlistdict)
    namelistdf = pd.DataFrame(namelist).rename(columns={0:'name'}).reset_index()
    df = pd.merge(namelistdf,df,on='index')
    df = df.drop(columns='index').drop(columns = 'Name').rename(columns={'name':'Name'}).drop(columns=['POS','HT','WT','Class','Birthplace'])
    print(teamid)
teamnameiddf['rosterlist'] = rosterlist
teamnameiddf['fullnamerosterlist'] = fullnamerosterlist
teamnameiddf['rosterlistdictionary'] = rosterlistdictlist
teamnameiddf.to_excel('teamnameidroster.xlsx')
display(teamnameiddf)

import json
newteamnameiddf = teamnameiddf
newteamnameiddf = newteamnameiddf.drop(columns = 'rosterlist').drop(columns = 'fullnamerosterlist').drop(columns = 'teamname')
display(newteamnameiddf)
teamidrosterdict = {}
for index,row in newteamnameiddf.iterrows():
    teamid = row['id']
    rosterdict = row['rosterlistdictionary']
    teamidrosterdict[teamid] = rosterdict
teamidrosterdict
with open('teamidrosterdict.json', 'w') as fp:
    json.dump(teamidrosterdict, fp)

#opens schedule of specific teamid
gameiddict1 = {}
for teamid in teamnameiddf['id']:
    gametimelist = []
    url = f'https://www.espn.com/mens-college-basketball/team/schedule/_/id/{teamid}'

    #scrapes all the urls on the website
    reqs = requests.get(url,headers=myheaders)
    soup = BeautifulSoup(reqs.text, 'html.parser')

    #get the times out of the links
    times = pd.read_html(url)
    times = times[0]
    times = times.rename(columns = {0:'Date',1:'Opponent',2:'Time'})

    # Get all the dates of the games
    for i,row in times.iterrows():
        line = str(times['Date'][i])
        if len(line) < 10 or len(line)> 11:
            times = times.drop(labels=[i],axis=0)
    times = times.reset_index().drop(columns='index').drop(columns=3).drop(columns=4).drop(columns=5).drop(columns=6).drop(columns=7).drop(columns = 'Opponent').drop(columns = 'Time')
    times = times.reset_index()
    # Add dates to list
    for date in times['Date']:
        gametimelist.append(date)
    i = 0
    #goes through each link on page
    for link in soup.find_all('a'):
        newlink = link.get('href')
        newlink = str(newlink)

        #keeps only the right links for games
        string = 'game/_/gameId/'

        #extracts gameid from the game links
        if string in newlink:
            gameid = newlink.split('/')[-1]

            #add game id to the list
            if gameid in gameiddict1.keys():
                teamid2 = {}
                teamid2['teamid2'] = teamid
                gameiddict1[f'{gameid}'] += [teamid2]
            else:
                teamid1 = {}
                teamid1['teamid1'] = teamid
                gameiddict1[f'{gameid}'] = [teamid1]
                gamedate = {}
                gamedate['date'] = gametimelist[i]
                gameiddict1[f'{gameid}'] += [gamedate]
            i = i + 1
            #print(f'team:{teamname} ---- id: {teamid}')

# clean up the data by getting rid of games with only 1 game id
import json
gameiddict2 = gameiddict1
for k in list(gameiddict2.keys()):
    if len(gameiddict2[k]) < 3:
        del gameiddict2[k]

niceprint = json.dumps(gameiddict2,indent=3)
print(len(gameiddict2))
print(niceprint)
#gameiddict2

newdict2 = pd.DataFrame.from_dict(gameiddict2).transpose().reset_index().rename(columns = {'index':'gameid',0:'teamID1',1:'date',2:'teamID2'})
teamid1list = []
teamid2list = []
datelist = []
for value in newdict2['teamID1']:
    value = value['teamid1']
    teamid1list.append(value)
for value in newdict2['teamID2']:
    value = value['teamid2']
    teamid2list.append(value)
for value in newdict2['date']:
    value = value['date']
    datelist.append(value)
newdict2['teamID1'] = teamid1list
newdict2['teamID2'] = teamid2list
newdict2['date'] = datelist
gameiddf = newdict2
display(newdict2)

gameiddfnew = pd.merge(gameiddf,teamnameiddf,left_on='teamID1',right_on='id',how='left')
gameiddfnew = gameiddfnew.rename(columns = {'teamname':'teamID1name','rosterlist':'teamID1roster'}).drop(columns = 'id')
gameiddfnew = pd.merge(gameiddfnew,teamnameiddf,left_on = 'teamID2',right_on='id',how='left')
gameiddfnew = gameiddfnew.rename(columns = {'teamname':'teamID2name','rosterlist':'teamID2roster'}).drop(columns = 'id')
gameiddfnew['gamename'] = gameiddfnew['teamID1name'] + ' vs ' + gameiddfnew['teamID2name']

gameIDdictionary = gameiddfnew.transpose()
new_header = gameIDdictionary.iloc[0] #grab the first row for the header
gameIDdictionary = gameIDdictionary[1:] #take the data less the header row
gameIDdictionary.columns = new_header #set the header row as the df header
display(gameIDdictionary)
gameIDdictionary = gameIDdictionary.to_dict()
import json

with open('gameIDdictionary.json', 'w') as fp:
    json.dump(gameIDdictionary, fp)

team:Boston College Eagles ---- id: 103
team:Boston College Eagles ---- id: 103
team:Clemson Tigers ---- id: 228
team:Clemson Tigers ---- id: 228
team:Duke Blue Devils ---- id: 150
team:Duke Blue Devils ---- id: 150
team:Florida State Seminoles ---- id: 52
team:Florida State Seminoles ---- id: 52
team:Georgia Tech Yellow Jackets ---- id: 59
team:Georgia Tech Yellow Jackets ---- id: 59
team:Louisville Cardinals ---- id: 97
team:Louisville Cardinals ---- id: 97
team:Miami Hurricanes ---- id: 2390
team:Miami Hurricanes ---- id: 2390
team:Nc State Wolfpack ---- id: 152
team:Nc State Wolfpack ---- id: 152
team:North Carolina Tar Heels ---- id: 153
team:North Carolina Tar Heels ---- id: 153
team:Notre Dame Fighting Irish ---- id: 87
team:Notre Dame Fighting Irish ---- id: 87
team:Pittsburgh Panthers ---- id: 221
team:Pittsburgh Panthers ---- id: 221
team:Syracuse Orange ---- id: 183
team:Syracuse Orange ---- id: 183
team:Virginia Cavaliers ---- id: 258
team:Virginia Cavaliers ---- id: 258
te

,teamname,id
0,Boston College Eagles,103
1,Clemson Tigers,228
2,Duke Blue Devils,150
3,Florida State Seminoles,52
4,Georgia Tech Yellow Jackets,59
...,...,...
357,Portland Pilots,2501
358,Saint Marys Gaels,2608
359,San Diego Toreros,301
360,San Francisco Dons,2539


103
228
150
52
59
97
2390
152
153
87
221
183
258
259
154
2046
91
2110
2198
526
294
338
288
2453
2454
2511
56
2066
2803
311
2885
160
399
2378
2349
261
2429
151
2226
235
249
242
2567
58
218
2655
202
5
2636
2724
2166
2168
2184
2230
2244
45
2325
2350
113
227
257
2603
139
179
2670
252
239
2132
248
66
2305
2306
201
197
2628
251
2641
2116
277
2086
156
305
46
269
2507
2550
2599
41
222
2752
331
304
70
149
147
2464
2458
2502
16
2692
2127
2241
2272
2344
2506
2515
2908
2427
2737
356
84
2294
120
127
130
135
158
77
194
213
2509
164
275
13
2934
2239
2463
62
299
302
300
27
28
2540
2097
232
48
2182
2210
2261
2275
2405
2448
111
2619
119
350
2729
2229
55
2335
2348
2393
166
2534
2638
98
325
2174
2739
85
270
94
2473
2870
2523
2750
2754
2130
225
171
172
159
108
219
163
43
2099
2217
314
2363
2368
116
315
2514
2520
2612
2561
2154
2169
47
2379
2415
2450
2428
2569
2006
2050
189
2084
2117
2199
2309
193
2459
195
2649
2711
2057
71
2181
339
2287
282
2623
93
2460
79
82
2674
2005
68
36
278
2440
167
21
23
2439
328
275

,teamname,id,rosterlist,fullnamerosterlist,rosterlistdictionary
0,Boston College Eagles,103,"[Aligbe, P, Atiyeh, A, Jr, Jr, Hastings, J, II...","[Prince Aligbe, Abe Atiyeh, Donald Hand Jr., C...","{'Aligbe, P': 'Prince Aligbe', 'Atiyeh, A': 'A..."
1,Clemson Tigers,228,"[Beadle, J, Clark, J, III, J, Godfrey, R, Hall...","[Josh Beadle, Jack Clark, Joseph Girard III, R...","{'Beadle, J': 'Josh Beadle', 'Clark, J': 'Jack..."
2,Duke Blue Devils,150,"[Begovich, N, Blakes, J, Borden, S, Filipowski...","[Neal Begovich, Jaylen Blakes, Stanley Borden,...","{'Begovich, N': 'Neal Begovich', 'Blakes, J': ..."
3,Florida State Seminoles,52,"[Adebisi, S, Bowen, T, Brown, M, Corhen, C, Fl...","[Sola Adebisi, Taylor Bol Bowen, Michael Brown...","{'Adebisi, S': 'Sola Adebisi', 'Bowen, T': 'Ta..."
4,Georgia Tech Yellow Jackets,59,"[Abram, A, Claude, T, Coleman, D, Dowuona, E, ...","[Amaree Abram, Tyzhaun Claude, Dallan Coleman,...","{'Abram, A': 'Amaree Abram', 'Claude, T': 'Tyz..."
...,...,...,...,...,...
357,Portland Pilots,2501,"[Austin, C, Delano, V, Dengdit, B, Gebbers, K,...","[Chris Austin, Vincent Delano, Bol Dengdit, Ke...","{'Austin, C': 'Chris Austin', 'Delano, V': 'Vi..."
358,Saint Marys Gaels,2608,"[Barrett, L, Bennett, C, Bradtke, J, Ducas, A,...","[Luke Barrett, Cade Bennett, Jensen Bradtke, A...","{'Barrett, L': 'Luke Barrett', 'Bennett, C': '..."
359,San Diego Toreros,301,"[Beniwal, N, Callas, E, Chammaa, J, Dahlke, D,...","[Neel Beniwal, Emmanuel Callas, Joey Chammaa, ...","{'Beniwal, N': 'Neel Beniwal', 'Callas, E': 'E..."
360,San Francisco Dons,2539,"[Beasley, R, Beasley, R, Bieker, J, Cioe, J, G...","[Robby Beasley, Ryan Beasley, Justin Bieker, J...","{'Beasley, R': 'Ryan Beasley', 'Bieker, J': 'J..."


,id,rosterlistdictionary
0,103,"{'Aligbe, P': 'Prince Aligbe', 'Atiyeh, A': 'A..."
1,228,"{'Beadle, J': 'Josh Beadle', 'Clark, J': 'Jack..."
2,150,"{'Begovich, N': 'Neal Begovich', 'Blakes, J': ..."
3,52,"{'Adebisi, S': 'Sola Adebisi', 'Bowen, T': 'Ta..."
4,59,"{'Abram, A': 'Amaree Abram', 'Claude, T': 'Tyz..."
...,...,...
357,2501,"{'Austin, C': 'Chris Austin', 'Delano, V': 'Vi..."
358,2608,"{'Barrett, L': 'Luke Barrett', 'Bennett, C': '..."
359,301,"{'Beniwal, N': 'Neel Beniwal', 'Callas, E': 'E..."
360,2539,"{'Beasley, R': 'Ryan Beasley', 'Bieker, J': 'J..."


5281
{
   "401576156": [
      {
         "teamid1": "103"
      },
      {
         "date": "Mon, Nov 6"
      },
      {
         "teamid2": "2217"
      }
   ],
   "401593809": [
      {
         "teamid1": "103"
      },
      {
         "date": "Fri, Nov 10"
      },
      {
         "teamid2": "2643"
      }
   ],
   "401593790": [
      {
         "teamid1": "103"
      },
      {
         "date": "Wed, Nov 15"
      },
      {
         "teamid2": "257"
      }
   ],
   "401581739": [
      {
         "teamid1": "103"
      },
      {
         "date": "Sat, Nov 18"
      },
      {
         "teamid2": "108"
      }
   ],
   "401581584": [
      {
         "teamid1": "103"
      },
      {
         "date": "Wed, Nov 22"
      },
      {
         "teamid2": "36"
      }
   ],
   "401581585": [
      {
         "teamid1": "103"
      },
      {
         "date": "Thu, Nov 23"
      },
      {
         "teamid2": "2350"
      }
   ],
   "401574558": [
      {
         "teamid1": "103

,gameid,teamID1,date,teamID2
0,401576156,103,"Mon, Nov 6",2217
1,401593809,103,"Fri, Nov 10",2643
2,401593790,103,"Wed, Nov 15",257
3,401581739,103,"Sat, Nov 18",108
4,401581584,103,"Wed, Nov 22",36
...,...,...,...,...
5276,401589697,301,"Thu, Feb 1",2539
5277,401589699,301,"Sat, Feb 3",2541
5278,401589718,301,"Sat, Feb 17",2541
5279,401589710,2539,"Sat, Feb 10",2541


gameid,401576156,401593809,401593790,401581739,401581584,401581585,401574558,401604089,401592106,401593810,...,401589686,401589696,401589719,401589723,401589674,401589697,401589699,401589718,401589710,401589734
teamID1,103,103,103,103,103,103,103,103,103,103,...,2608,2608,2608,2608,301,301,301,301,2539,2539
date,"Mon, Nov 6","Fri, Nov 10","Wed, Nov 15","Sat, Nov 18","Wed, Nov 22","Thu, Nov 23","Wed, Nov 29","Sat, Dec 2","Tue, Dec 5","Fri, Dec 8",...,"Sat, Jan 20","Wed, Jan 31","Tue, Feb 20","Sat, Feb 24","Thu, Jan 11","Thu, Feb 1","Sat, Feb 3","Sat, Feb 17","Sat, Feb 10","Sat, Mar 2"
teamID2,2217,2643,257,108,36,2350,238,152,2115,107,...,2539,2541,2539,301,2539,2539,2541,2541,2541,2541
teamID1name,Boston College Eagles,Boston College Eagles,Boston College Eagles,Boston College Eagles,Boston College Eagles,Boston College Eagles,Boston College Eagles,Boston College Eagles,Boston College Eagles,Boston College Eagles,...,Saint Marys Gaels,Saint Marys Gaels,Saint Marys Gaels,Saint Marys Gaels,San Diego Toreros,San Diego Toreros,San Diego Toreros,San Diego Toreros,San Francisco Dons,San Francisco Dons
teamID1roster,"[Aligbe, P, Atiyeh, A, Jr, Jr, Hastings, J, II...","[Aligbe, P, Atiyeh, A, Jr, Jr, Hastings, J, II...","[Aligbe, P, Atiyeh, A, Jr, Jr, Hastings, J, II...","[Aligbe, P, Atiyeh, A, Jr, Jr, Hastings, J, II...","[Aligbe, P, Atiyeh, A, Jr, Jr, Hastings, J, II...","[Aligbe, P, Atiyeh, A, Jr, Jr, Hastings, J, II...","[Aligbe, P, Atiyeh, A, Jr, Jr, Hastings, J, II...","[Aligbe, P, Atiyeh, A, Jr, Jr, Hastings, J, II...","[Aligbe, P, Atiyeh, A, Jr, Jr, Hastings, J, II...","[Aligbe, P, Atiyeh, A, Jr, Jr, Hastings, J, II...",...,"[Barrett, L, Bennett, C, Bradtke, J, Ducas, A,...","[Barrett, L, Bennett, C, Bradtke, J, Ducas, A,...","[Barrett, L, Bennett, C, Bradtke, J, Ducas, A,...","[Barrett, L, Bennett, C, Bradtke, J, Ducas, A,...","[Beniwal, N, Callas, E, Chammaa, J, Dahlke, D,...","[Beniwal, N, Callas, E, Chammaa, J, Dahlke, D,...","[Beniwal, N, Callas, E, Chammaa, J, Dahlke, D,...","[Beniwal, N, Callas, E, Chammaa, J, Dahlke, D,...","[Beasley, R, Beasley, R, Bieker, J, Cioe, J, G...","[Beasley, R, Beasley, R, Bieker, J, Cioe, J, G..."
fullnamerosterlist_x,"[Prince Aligbe, Abe Atiyeh, Donald Hand Jr., C...","[Prince Aligbe, Abe Atiyeh, Donald Hand Jr., C...","[Prince Aligbe, Abe Atiyeh, Donald Hand Jr., C...","[Prince Aligbe, Abe Atiyeh, Donald Hand Jr., C...","[Prince Aligbe, Abe Atiyeh, Donald Hand Jr., C...","[Prince Aligbe, Abe Atiyeh, Donald Hand Jr., C...","[Prince Aligbe, Abe Atiyeh, Donald Hand Jr., C...","[Prince Aligbe, Abe Atiyeh, Donald Hand Jr., C...","[Prince Aligbe, Abe Atiyeh, Donald Hand Jr., C...","[Prince Aligbe, Abe Atiyeh, Donald Hand Jr., C...",...,"[Luke Barrett, Cade Bennett, Jensen Bradtke, A...","[Luke Barrett, Cade Bennett, Jensen Bradtke, A...","[Luke Barrett, Cade Bennett, Jensen Bradtke, A...","[Luke Barrett, Cade Bennett, Jensen Bradtke, A...","[Neel Beniwal, Emmanuel Callas, Joey Chammaa, ...","[Neel Beniwal, Emmanuel Callas, Joey Chammaa, ...","[Neel Beniwal, Emmanuel Callas, Joey Chammaa, ...","[Neel Beniwal, Emmanuel Callas, Joey Chammaa, ...","[Robby Beasley, Ryan Beasley, Justin Bieker, J...","[Robby Beasley, Ryan Beasley, Justin Bieker, J..."
rosterlistdictionary_x,"{'Aligbe, P': 'Prince Aligbe', 'Atiyeh, A': 'A...","{'Aligbe, P': 'Prince Aligbe', 'Atiyeh, A': 'A...","{'Aligbe, P': 'Prince Aligbe', 'Atiyeh, A': 'A...","{'Aligbe, P': 'Prince Aligbe', 'Atiyeh, A': 'A...","{'Aligbe, P': 'Prince Aligbe', 'Atiyeh, A': 'A...","{'Aligbe, P': 'Prince Aligbe', 'Atiyeh, A': 'A...","{'Aligbe, P': 'Prince Aligbe', 'Atiyeh, A': 'A...","{'Aligbe, P': 'Prince Aligbe', 'Atiyeh, A': 'A...","{'Aligbe, P': 'Prince Aligbe', 'Atiyeh, A': 'A...","{'Aligbe, P': 'Prince Aligbe', 'Atiyeh, A': 'A...",...,"{'Barrett, L': 'Luke Barrett', 'Bennett, C': '...","{'Barrett, L': 'Luke Barrett', 'Bennett, C': '...","{'Barrett, L': 'Luke Barrett', 'Bennett, C': '...","{'Barrett, L': 'Luke Barrett', 'Bennett, C': '...","{'Beniwa